In [1]:
import numpy as np
from numpy.linalg import multi_dot
from numpy.linalg import qr
import numpy.linalg as npla
import scipy as sp
from scipy.linalg import block_diag, logm, eigvals
from multiprocessing import Pool
from functools import partial
import multiprocessing as mp
from multiprocessing import set_start_method
from multiprocessing import get_context
import joblib
import time
import matplotlib.pyplot as plt
from scipy.linalg import lu

theta_crit = np.pi/4
phases = np.array([0.5,0.5,0.5])

In [5]:
strip_width = 32
strip_length = 1000
theta = theta_crit
LU = 'LU'
QR = 'QR'

def LU_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, LU)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def LU_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, LU)
    end_time = time.process_time() - start_time
    return float(end_time)


In [5]:
strip_width = 32
strip_length = 1000
theta = theta_crit
LU = 'LU'
QR = 'QR'

def LU_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, LU)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def LU_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, LU)
    end_time = time.process_time() - start_time
    return float(end_time)


In [2]:
def FullStrip(sin,cos,strip_width):
        
        def TAS(sin,cos,strip_width):
        
            def TAMatrix(sin,cos, phi_one,phi_two,phi_three):
                matrix_one = np.array([[1,0],[0,np.exp(-1j*phi_three)]])
                matrix_two = np.array([[1,-sin],[-sin,1]])
                matrix_three = np.array([[np.exp(1j*phi_two),0],[0,np.exp(-1j*phi_one)]])

                return (1/cos)*multi_dot([matrix_one,matrix_two,matrix_three])

            Tslist = [ 
                TAMatrix(sin,cos,*(2*np.pi)*phases) for i in range(strip_width)
            ]
            return block_diag(*Tslist)

        def TBS(sin,cos,strip_width):

            def TBMatrix(sin,cos, phi_one,phi_two,phi_three):   
                matrix_one = np.array([[np.exp(-1j*phi_three),0],[0,np.exp(1j*phi_one)]])
                matrix_two = np.array([[1,cos],[cos,1]])
                matrix_three = np.array([[-1,0],[0,np.exp(1j*phi_two)]])

                return (1/sin*multi_dot([matrix_one,matrix_two,matrix_three]))

            Tslist = [ 
                TBMatrix(sin,cos,*(2*np.pi)*phases) for i in range(strip_width-1)
            ]
            extra = TBMatrix(sin,cos,*(2*np.pi)*phases)
            temp_mat = block_diag(extra[1,1],*Tslist,extra[0,0])    
            temp_mat[0,(2*strip_width)-1] = extra[1,0]
            temp_mat[(2*strip_width)-1,0] = extra[0,1]
            return temp_mat
    
        return np.matmul(TAS(sin,cos,strip_width),TBS(sin,cos,strip_width))

In [3]:
def FullTransfer(strip_length,strip_width,theta,group_val, method): 
    sin = np.sin(theta)
    cos = np.cos(theta)
    
    group_val = int(group_val)
    
    matricies = [FullStrip(sin,cos,strip_width) for i in range(strip_length)]
    
    if group_val > 1:
        every_nth = [multi_dot(matricies[i:i+group_val]) for i in range(int(strip_length/group_val))]

        if method == 'LU':
            Tone = matricies[0]
            pone,lone,uone = lu(Tone)
            bigQ = np.matmul(pone,lone)
            rlog_one = np.log(np.absolute(uone.diagonal()))
            for i in every_nth:
                matrixb = np.matmul(i,bigQ)
                p,l,u = lu(matrixb)
                bigQ = np.matmul(p,l)
                rlogs = np.log(np.absolute(u.diagonal()))
                rlog_one = np.add(rlogs,rlog_one)
            return rlog_one

        if method == 'QR' :
            Tone = matricies[0]
            qone,rone = qr(Tone)
            bigQ = qone
            rlog_one = np.log(np.absolute(rone.diagonal()))
            for i in every_nth:
                matrixb = np.matmul(i,bigQ)
                q,r = qr(matrixb)
                bigQ = q
                rlogs = np.log(np.absolute(r.diagonal()))
                rlog_one = np.add(rlogs,rlog_one)
            return rlog_one
    else:
        if method == 'LU':
            Tone = matricies[0]
            pone,lone,uone = lu(Tone)
            bigQ = np.matmul(pone,lone)
            rlog_one = np.log(np.absolute(uone.diagonal()))
            for i in matricies:
                matrixb = np.matmul(i,bigQ)
                p,l,u = lu(matrixb)
                bigQ = np.matmul(p,l)
                rlogs = np.log(np.absolute(u.diagonal()))
                rlog_one = np.add(rlogs,rlog_one)
            return rlog_one

        if method == 'QR' :
            Tone = matricies[0]
            qone,rone = qr(Tone)
            bigQ = qone
            rlog_one = np.log(np.absolute(rone.diagonal()))
            for i in matricies:
                matrixb = np.matmul(i,bigQ)
                q,r = qr(matrixb)
                bigQ = q
                rlogs = np.log(np.absolute(r.diagonal()))
                rlog_one = np.add(rlogs,rlog_one)
            return rlog_one

In [5]:
strip_width = 32
strip_length = 1000
theta = theta_crit
LU = 'LU'
QR = 'QR'

def LU_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, LU)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_every_nth_timing(strip_width,strip_length,theta,group_val, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,group_val, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def QR_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, QR)
    end_time = time.process_time() - start_time
    return float(end_time)

def LU_timing(strip_width,strip_length,theta, rand):
    rand = rand
    start_time = time.process_time()
    FullTransfer(strip_length,strip_width,theta,1, LU)
    end_time = time.process_time() - start_time
    return float(end_time)


In [13]:
#LU_every_nth_timing(strip_width,strip_length,theta,3)
#QR_every_nth_timing(strip_width,strip_length,theta,3)
#QR_timing(strip_width,strip_length,theta)
#LU_timing(strip_width,strip_length,theta)

4.393155097961426

In [10]:
range_val = 3500



start_time = time.time()
p = get_context("fork").Pool()

'''
QR_partial_function = partial(QR_timing,strip_width,strip_length,theta)
LU_partial_function = partial(LU_timing,strip_width,strip_length,theta)

QR_time = (p.map(QR_partial_function,[i for i in range(range_val)]))
joblib.dump(QR_time, 'Timing_Data/QR_every_1.pkl')
LU_time = (p.map(LU_partial_function,[i for i in range(range_val)]))
joblib.dump(LU_time, 'Timing_Data/LU_every_1.pkl')
'''
for i in range(5,21):
    #LU_nth_partial_function = partial(LU_every_nth_timing,strip_width,strip_length,theta, i)
    QR_nth_partial_function = partial(QR_every_nth_timing,strip_width,strip_length,theta, i)

    #LUEverynth_time = (p.map(LU_nth_partial_function,[i for i in range(range_val)]))
    #joblib.dump(LUEverynth_time, 'Timing_Data/LU_every_%s.pkl' %(i))
    QREverynth_time = (p.map(QR_nth_partial_function,[i for i in range(range_val)]))
    joblib.dump(QREverynth_time, 'Timing_Data/QR_every_%s.pkl' %(i))


print('%s minutes'%((time.time() - start_time)/60))

1047.0440919319788 minutes


In [56]:
#value validation creation

validation_partial = partial(FullTransfer,strip_length,strip_width,theta)
params = [(5,LU),(5,QR),(1,LU),(1,QR)]
validation_data_sets=[validation_partial(i[0],i[1]) for i in params]

#value testing
correct_set = validation_data_sets[3]
for j in range(3):
    error = [abs(validation_data_sets[j][i] - correct_set[i]) for i in range(len(correct_set))]
    print('-- Max = %s -- Mean Err = %s -- StDev = %s --'%(max(error),np.mean(error),np.std(error)))

-- Max = 2.4047645146084213 -- Mean Err = 0.32847114716250125 -- StDev = 0.3275117117253104 --
-- Max = 1.6570744776345236e-11 -- Mean Err = 5.856717021079927e-13 -- StDev = 2.7721055539849012e-12 --
-- Max = 2.0909094083493 -- Mean Err = 0.32843309136809506 -- StDev = 0.3255683940838234 --


In [21]:
6*60/3.7*40

3891.8918918918916

In [26]:
0.3/4.62

0.06493506493506493

In [4]:
390/3

130.0